In [36]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

def makechart():
    table = pd.read_csv("price.csv")
    table = table[table["Price"] > 100]
    table["Date"] = pd.to_datetime(table["Date"])
    table["Street"] = table["Street"].str.replace("  ", " ")
    table["last word"] = table["Street"].str.split().str[-1]

   
    freq = table["last word"].value_counts()
    table = table.join(freq, on="last word", rsuffix="_count")
    table.loc[table["count"] < 50, "last word"] = "OTHER"
    table = table.sort_values(by="count", ascending=False)
    table["last word"] = table["last word"].astype(str)

   
    grouped = (table[["last word", "Price"]].groupby("last word").agg(["count", "mean", "min", "max", "std", "median"]).reset_index())
    grouped.columns = ["last word", "count", "mean", "min", "max", "std", "median"]
    grouped = grouped[grouped["last word"] != "nan"]

   
    grouped["maxdiff"] = grouped["max"] - grouped["median"]
    grouped["mindiff"] = grouped["median"] - grouped["min"]
    grouped=grouped.sort_values(by="last word", ascending=False).reset_index(drop=True) 
  
    fig = px.scatter(grouped,x="median",y="last word",error_x="maxdiff",error_x_minus="mindiff",height=3300,width=1000,log_x=True,labels={"median":"Price","last word":"Last word in street name"})
    fig.add_trace(px.scatter(grouped,x="mean",y="last word",color_discrete_sequence=["#001a3a"],).data[0])
    fig.add_trace(px.bar(grouped,x="count",y="last word",color_discrete_sequence=["#c2c3c4"]).update_traces(width=0.5).data[0])

  


    fig.update_layout(
        xaxis_side="top",
        xaxis_title="Price",
        yaxis_title="Last word in street name",
        yaxis=dict(tickfont=dict(size=8)),
        legend=dict(title="Series", orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
    
 
    #if len(fig.data) >= 3:
    fig.data[0].name = "Median Price"
    fig.data[0].showlegend = True
    fig.data[1].name = "Mean Price"
    fig.data[1].showlegend = True
    fig.data[2].name = "Popularity"
    fig.data[2].showlegend = True

    print(grouped.columns)
    grouped.columns=["Street Suffix","Popularity","Mean Price","Min Price","Max Price","St. Dev","Median Price","maxdiff","mindiff"]
    buttons = []
    for key in ["Median Price", "Mean Price", "Min Price", "Max Price","Popularity","Street Suffix"]:
        ascending = key not in ["Min Price", "Max Price", "Street Suffix"]  # sort min/max descending
        grouped = grouped.sort_values(by=key, ascending=ascending).reset_index(drop=True)

        y_order = grouped["Street Suffix"].astype(str).tolist()
        x_median = grouped["Median Price"].tolist()
        x_mean = grouped["Mean Price"].tolist()
        x_count = grouped["Popularity"].tolist()
        maxdiff = grouped["maxdiff"].tolist()
        mindiff = grouped["mindiff"].tolist()

        buttons.append(
            dict(
                label=f"Sort by {key}",
                method="update",
                args=[
                    {"x": [x_median, x_mean, x_count],"y": [y_order, y_order, y_order],"error_x": [dict(type="data",array=maxdiff,arrayminus=mindiff,visible=True),None,None]},
                    {"yaxis": dict(type="category",categoryorder="array",categoryarray=y_order,tickfont=dict(size=8))}
                    ]))
       


    fig.update_layout(updatemenus=[dict(buttons=buttons,direction="down",showactive=True)])

    fig.write_html("graph.html")
    fig.show()

makechart()


Index(['last word', 'count', 'mean', 'min', 'max', 'std', 'median', 'maxdiff',
       'mindiff'],
      dtype='object')
